In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import spotipy
import spotipy.util as util
import json
import time
import urllib.request

import pandas as pd

In [4]:
def generate_token():
    with open('configs.secret') as f:
        data = json.load(f)

    #scope = 'user-library-read'
    scope = None
    username = data['username']
    client_id = data['client_id']
    client_secret = data['client_secret']
    redirect_uri = data['redirect_uri']

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if not token:
        raise RuntimeError("Can't get token for", username)
    
    return token

In [5]:
token = generate_token()
sp = spotipy.Spotify(auth=token)

In [6]:
playlists = []
num_playlists = 800
limit = 50

for i in range(0, int(num_playlists / limit)):
    results = sp.search('Banger', limit=limit, offset=limit*i, type='playlist')
    filtered_playlists = filter(lambda x: 'banger' in x['name'].lower(), results['playlists']['items'])
    playlists.extend(filtered_playlists)
    time.sleep(0.1)

print('Total filtered playlists', len(playlists))

Total filtered playlists 758


In [7]:
tracks = {}

# Loop through playlists
for playlist in playlists:
    
    # A track can be added to a playlist several times. Only count one occurance per playlist
    
    playlist_track_deduper = []
    # If more than 100 songs in playlist, requires multiple queries to get all songs
    next = True
    offset = 0
    while (next):
        results = sp.playlist_tracks(playlist['id'],offset=offset)
        #print(playlist['uri'],len(results['items']))
        for item in results['items']:
            track = item['track']
            
            if not track:
                print(item)
                continue
            
            # Ignore tracks that are not on Spotify
            if track['is_local']:
                continue
                
            track_id = track['id']
            
            # Only count a single occurance of a track per playlist
            if track_id in playlist_track_deduper:
                continue
            playlist_track_deduper.append(track_id)
            
            # increment occurances if present
            if track_id in tracks:
                tracks[track_id]['banger_occurances'] = tracks[track_id]['banger_occurances'] + 1
            
            # else create new track entry
            else:        
                track.pop('album', None)
                track.pop('available_markets', None)
                track['banger_occurances'] = 1
                tracks[track_id] = track
                
        offset += 100
        next = results['next']
        time.sleep(0.1)

print(len(tracks))
print(max(tracks, key=lambda x: tracks[x]['banger_occurances']))

{'added_at': '2017-11-17T12:50:26Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/rickkve'}, 'href': 'https://api.spotify.com/v1/users/rickkve', 'id': 'rickkve', 'type': 'user', 'uri': 'spotify:user:rickkve'}, 'is_local': False, 'primary_color': None, 'track': None, 'video_thumbnail': {'url': None}}
70535
2xLMifQCjDGFmkHkpNLD9h


In [8]:
top_track = max(tracks, key=lambda x: tracks[x]['banger_occurances'])
max_occurances = tracks[top_track]['banger_occurances']

print(top_track, max_occurances)

2xLMifQCjDGFmkHkpNLD9h 295


In [9]:
filtered_tracks = {k: v for k, v in tracks.items() if v['banger_occurances'] > 5}
print(len(tracks),len(filtered_tracks))

70535 5933


In [10]:
track_attributes_to_remove = ['codestring', 'code_version', 'echoprintstring', 'echoprint_version', 'synchstring', 'synch_version', 'rhythmstring', 'rhythm_version']
if False:
    # analysis: track, sections, segments,
    for track_id, track_details in filtered_tracks.items():
        try:
            results = sp.audio_analysis(track_id)

        # Takes long enough token can expire, refresh token
        except Exception as excpt:
            print(excpt)
            token = generate_token()
            sp = spotipy.Spotify(auth=token)
            results = sp.audio_analysis(track_id)

        track_info = results['track']
        for key in track_attributes_to_remove:
            track_info.pop(key, None)
        tmp_track_details = {**track_details, **track_info}
        tmp_track_details['sections'] = results['sections']

        filtered_tracks[track_id] = tmp_track_details

In [11]:
track_ids = list(filtered_tracks.keys())
num_tracks = len(track_ids)
num_slices = int(num_tracks / 50)
for i in range(num_slices):
    try:
        results = sp.audio_features(track_ids[i * 50 : i * 50 + 50])
    
    # Takes long enough token can expire, refresh token
    except Exception as excpt:
        token = generate_token()
        sp = spotipy.Spotify(auth=token)
        results = sp.audio_features(track_ids[i * 50 : i * 50 + 50])
    for j in range(50):
        track_id = track_ids[i * 50 + j]
        track_details = filtered_tracks[track_id]
        tmp_track_details = {**track_details, **results[j]}
        filtered_tracks[track_id] = tmp_track_details
    time.sleep(0.1)
        

In [12]:
# artist.hotttnesss,artist.id,artist.name,artist_mbtags,artist_mbtags_count,bars_confidence,bars_start,beats_confidence,beats_start,duration,end_of_fade_in,familiarity,key,key_confidence,latitude,location,longitude,loudness,mode,mode_confidence,release.id,release.name,similar,song.hotttnesss,song.id,start_of_fade_out,tatums_confidence,tatums_start,tempo,terms,terms_freq,time_signature,time_signature_confidence,title,year


In [13]:
print(filtered_tracks['2xLMifQCjDGFmkHkpNLD9h'])

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0Y5tJX1MQlPlqiwlOH1tJY'}, 'href': 'https://api.spotify.com/v1/artists/0Y5tJX1MQlPlqiwlOH1tJY', 'id': '0Y5tJX1MQlPlqiwlOH1tJY', 'name': 'Travis Scott', 'type': 'artist', 'uri': 'spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY'}], 'disc_number': 1, 'duration_ms': 312820, 'episode': False, 'explicit': True, 'external_ids': {'isrc': 'USSM11806660'}, 'external_urls': {'spotify': 'https://open.spotify.com/track/2xLMifQCjDGFmkHkpNLD9h'}, 'href': 'https://api.spotify.com/v1/tracks/2xLMifQCjDGFmkHkpNLD9h', 'id': '2xLMifQCjDGFmkHkpNLD9h', 'is_local': False, 'name': 'SICKO MODE', 'popularity': 94, 'preview_url': 'https://p.scdn.co/mp3-preview/1c3c01c64477b356e28a652b6447f4ef96689a71?cid=b48aa986c53946d3b37781455e256dd4', 'track': True, 'track_number': 3, 'type': 'audio_features', 'uri': 'spotify:track:2xLMifQCjDGFmkHkpNLD9h', 'banger_occurances': 295, 'danceability': 0.834, 'energy': 0.73, 'key': 8, 'loudness': -3.714, 'mode': 1, '